In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [2]:
url_inicial = 'http://www.revistas.unam.mx/index.php/rie/issue/archive'
url_root = 'http://www.revistas.unam.mx/index.php/rie/issue/archive'
r=requests.get(url_inicial)

In [3]:
soup = BeautifulSoup(r.text, 'html.parser')

In [4]:
box = soup.find('div', id='content')
volumen=box.findAll('h4')
vol = [x.find('a').get('href')for x in volumen]
vol=[urljoin(url_root,i) for i in vol]
vol

['http://www.revistas.unam.mx/index.php/rie/issue/view/5985',
 'http://www.revistas.unam.mx/index.php/rie/issue/view/5984',
 'http://www.revistas.unam.mx/index.php/rie/issue/view/5983',
 'http://www.revistas.unam.mx/index.php/rie/issue/view/5832',
 'http://www.revistas.unam.mx/index.php/rie/issue/view/5831',
 'http://www.revistas.unam.mx/index.php/rie/issue/view/5830',
 'http://www.revistas.unam.mx/index.php/rie/issue/view/5829',
 'http://www.revistas.unam.mx/index.php/rie/issue/view/5358',
 'http://www.revistas.unam.mx/index.php/rie/issue/view/5357',
 'http://www.revistas.unam.mx/index.php/rie/issue/view/5356',
 'http://www.revistas.unam.mx/index.php/rie/issue/view/5355',
 'http://www.revistas.unam.mx/index.php/rie/issue/view/5128',
 'http://www.revistas.unam.mx/index.php/rie/issue/view/5127',
 'http://www.revistas.unam.mx/index.php/rie/issue/view/5126',
 'http://www.revistas.unam.mx/index.php/rie/issue/view/5125',
 'http://www.revistas.unam.mx/index.php/rie/issue/view/4897',
 'http:/

In [5]:
vol2 =[]
for i in vol: 
    url_inicial1=i 
    r1 = requests.get(url_inicial1)
    soup1 = BeautifulSoup(r1.text, 'html.parser')
    box1 = soup1.find('div', id='content') 
    volumen1=box1.findAll('table', class_='tocArticle')
    vol1 = [x.find('a').get('href')for x in volumen1]
    vol2+=vol1

In [6]:
vol2

['http://www.revistas.unam.mx/index.php/rie/article/view/81476',
 'http://www.revistas.unam.mx/index.php/rie/article/view/82097',
 'http://www.revistas.unam.mx/index.php/rie/article/view/81800',
 'http://www.revistas.unam.mx/index.php/rie/article/view/79688',
 'http://www.revistas.unam.mx/index.php/rie/article/view/81694',
 'http://www.revistas.unam.mx/index.php/rie/article/view/82062',
 'http://www.revistas.unam.mx/index.php/rie/article/view/81564',
 'http://www.revistas.unam.mx/index.php/rie/article/view/82246',
 'http://www.revistas.unam.mx/index.php/rie/article/view/82247',
 'http://www.revistas.unam.mx/index.php/rie/article/view/81156',
 'http://www.revistas.unam.mx/index.php/rie/article/view/81132',
 'http://www.revistas.unam.mx/index.php/rie/article/view/79285',
 'http://www.revistas.unam.mx/index.php/rie/article/view/79238',
 'http://www.revistas.unam.mx/index.php/rie/article/view/81521',
 'http://www.revistas.unam.mx/index.php/rie/article/view/79901',
 'http://www.revistas.una

In [7]:
def get_url_items(sopa,url):
    box1 = soup1.find('div', id='content') 
    volumen1=box1.findAll('table', class_='tocArticle')
    vol1 = [x.find('a').get('href')for x in volumen1]
    return vol2

In [8]:
links_items=[]
i=0
while i<1:
    i+=1
    print(f'Estoy en la pagina {url_inicial}')
    r_pag=requests.get(url_inicial)
    s_p=BeautifulSoup(r_pag.text,'html.parser')
    links=get_url_items(s_p, url_inicial)
    links_items.append(links) 
    next_a=s_p.select('li.next > a')
    if not next_a or not next_a[0].get('href'):
        break

Estoy en la pagina http://www.revistas.unam.mx/index.php/rie/issue/archive


In [9]:
list_scraper=[]
for i in links_items:
    for j in i:
        list_scraper.append(j)
len(list_scraper)

371

In [10]:
uno=list_scraper[0]
r_item=requests.get(uno)
s_item=BeautifulSoup(r_item.text, 'html.parser')

In [11]:
#funcion para iniciar el escraper de cada libro
def scraper_book(url):
    content_book={}
    r=requests.get(url)
    tire='Investigación Económica'
    a='V. Ciencias Sociales'
    tem='Teorías Económicas y su aplicación'
    s_item=BeautifulSoup(r.text,'html.parser')
 #titulo de revista
    try:
        titulo=tire
        content_book['Titulo Revista']=tire
    except AttributeError:
        content_book['Titulo Revista']=None
    #area
    try:
        area=a
        content_book['Area']=a
    except AttributeError:
        content_book['Area']=None
    #tematica
    try:
        tema=tem
        content_book['Tematica']=tem
    except AttributeError:
        content_book['Tematica']=None
    #titulo articulo
    try:
        titu=s_item.find('div', id='articleTitle').get_text(strip=True)
        content_book['Titulo Articulo']=titu
    except AttributeError:
        content_book['Titulo Articulo']=None
#resumen
    try:
        resu=s_item.find('div', id='articleAbstract').text.lower().split("resumen")
        if len(resu) > 1:
            resu2 = resu[-1].split("abstract")[0].strip()
            content_book['Resumen']=resu2.replace("resumen","")
        else:
            content_book['Resumen']=None
    except AttributeError:
        content_book['Resumen']=None
    #abstract
    try:
        abst=s_item.find('div', id='articleAbstract').text.lower().split("abstract")
        #print(abst)
        if len(abst) > 1:
            abst2 = abst[-1].split("resumen")[0].strip()
            #print(abst2)
            content_book['Abstract']=abst2.replace("abstract","").strip()
        else:
            content_book['Abstract']=None
    except AttributeError:
        content_book['Abstract']=None
    #link artitulo
    try:
        linkart=url
        content_book['Link Articulo']=linkart
    except AttributeError:
        content_book['Link Articulo']=None
    #link articulo
    try:
        link=s_item.find('a', class_='file').get('href')
        content_book['Link PDF']=urljoin(url_root, link)
    except AttributeError:
        content_book['Link PDF']=None
    return content_book

In [12]:
list_scraper=list_scraper[0:371]
datos_book=[]
for idx, i in enumerate(list_scraper):
    print(f'estas escrapeando la pag {idx}')
    datos_book.append(scraper_book(i))

estas escrapeando la pag 0
estas escrapeando la pag 1
estas escrapeando la pag 2
estas escrapeando la pag 3
estas escrapeando la pag 4
estas escrapeando la pag 5
estas escrapeando la pag 6
estas escrapeando la pag 7
estas escrapeando la pag 8
estas escrapeando la pag 9
estas escrapeando la pag 10
estas escrapeando la pag 11
estas escrapeando la pag 12
estas escrapeando la pag 13
estas escrapeando la pag 14
estas escrapeando la pag 15
estas escrapeando la pag 16
estas escrapeando la pag 17
estas escrapeando la pag 18
estas escrapeando la pag 19
estas escrapeando la pag 20
estas escrapeando la pag 21
estas escrapeando la pag 22
estas escrapeando la pag 23
estas escrapeando la pag 24
estas escrapeando la pag 25
estas escrapeando la pag 26
estas escrapeando la pag 27
estas escrapeando la pag 28
estas escrapeando la pag 29
estas escrapeando la pag 30
estas escrapeando la pag 31
estas escrapeando la pag 32
estas escrapeando la pag 33
estas escrapeando la pag 34
estas escrapeando la pag 35
es

In [13]:
df_catalogo=pd.DataFrame(datos_book)
df_catalogo

,Titulo Revista,Area,Tematica,Titulo Articulo,Resumen,Abstract,Link Articulo,Link PDF
0,Investigación Económica,V. Ciencias Sociales,Teorías Económicas y su aplicación,THE LONG RECESSION AND ECONOMIC CONSEQUENCES O...,en este artículo argumentamos que la tasa de g...,"in this article, we argue the rate of profit i...",http://www.revistas.unam.mx/index.php/rie/arti...,http://www.revistas.unam.mx/index.php/rie/arti...
1,Investigación Económica,V. Ciencias Sociales,Teorías Económicas y su aplicación,THE GREEK CRISIS IN RETROSPECT: THE RATE OF PR...,la economía de grecia está en crisis desde el ...,the greek economy is in crisis since the fourt...,http://www.revistas.unam.mx/index.php/rie/arti...,http://www.revistas.unam.mx/index.php/rie/arti...
2,Investigación Económica,V. Ciencias Sociales,Teorías Económicas y su aplicación,"REDISTRIBUTION, INEQUALITY, AND GROWTH: GVAR A...",en este artículo se investiga con un modelo gv...,this article investigates the effect of gdp re...,http://www.revistas.unam.mx/index.php/rie/arti...,http://www.revistas.unam.mx/index.php/rie/arti...
3,Investigación Económica,V. Ciencias Sociales,Teorías Económicas y su aplicación,TRADE AGREEMENT AND TRADE SPECIALIZATION BETWE...,el estudio analiza la relevancia del acuerdo c...,the study analyses the relevance of the trade ...,http://www.revistas.unam.mx/index.php/rie/arti...,http://www.revistas.unam.mx/index.php/rie/arti...
4,Investigación Económica,V. Ciencias Sociales,Teorías Económicas y su aplicación,LA INVERSIÓN EXTRANJERA DIRECTA EN MÉXICO: ANÁ...,en este artículo se presenta un estudio empíri...,an empirical study of foreign direct investmen...,http://www.revistas.unam.mx/index.php/rie/arti...,http://www.revistas.unam.mx/index.php/rie/arti...
...,...,...,...,...,...,...,...,...
366,Investigación Económica,V. Ciencias Sociales,Teorías Económicas y su aplicación,EFECTOS DE LA LIBERACIÓN COMERCIAL EN EL CRECI...,se utilizan técnicas paramétricas y no paramét...,None,http://www.revistas.unam.mx/index.php/rie/arti...,http://www.revistas.unam.mx/index.php/rie/arti...
367,Investigación Económica,V. Ciencias Sociales,Teorías Económicas y su aplicación,THE CONCEPT OF EXPLOITATION IN A GENERAL LINEA...,en este trabajo consideramos el concepto de ex...,None,http://www.revistas.unam.mx/index.php/rie/arti...,http://www.revistas.unam.mx/index.php/rie/arti...
368,Investigación Económica,V. Ciencias Sociales,Teorías Económicas y su aplicación,LAS INTERRELACIONES DE VOLATILIDAD Y RENDIMIEN...,la liberalización y desregulación financieras ...,None,http://www.revistas.unam.mx/index.php/rie/arti...,http://www.revistas.unam.mx/index.php/rie/arti...
369,Investigación Económica,V. Ciencias Sociales,Teorías Económicas y su aplicación,ESTRUCTURA JERÁRQUICA Y DINÁMICA EN LOS MERCAD...,en este trabajo estudiamos la estructura jerár...,None,http://www.revistas.unam.mx/index.php/rie/arti...,http://www.revistas.unam.mx/index.php/rie/arti...


In [14]:
df_catalogo.to_csv('Revista061.csv', index=False)